In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Integrate and manage Public Transport data").getOrCreate()

In [0]:
spark.conf.set(
    f"fs.azure.account.key.omardbstorageaccount.dfs.core.windows.net", 
    "n/M6dGAvjc8505kkOdZWCcfky+UKdckTs1aAhVFqb/J7Ck1yTg+meYzFXNAb/oz1mxXbizdVQdB5+ASt8wSp4w=="
)


In [0]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Définition de la liste des mois et du nombre de jours dans chaque mois
months = [("January", 31), ("February", 28), ("March", 31), ("April", 30), ("May", 31)]

# Montage du lac de données
storageAccountName = "omardbstorageaccount"
storageAccountAccessKey = "n/M6dGAvjc8505kkOdZWCcfky+UKdckTs1aAhVFqb/J7Ck1yTg+meYzFXNAb/oz1mxXbizdVQdB5+ASt8wSp4w=="
sasToken = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2023-09-28T15:43:37Z&st=2023-09-27T07:43:37Z&spr=https&sig=13kFGISyoUEheYmR4g9YC3qa4hCoO8YDOEQzIorKfCw%3D"
blobContainerName = "public-transport-data"
mountPoint = "/mnt/public-transport-data/"

# Vérification si le montage existe déjà
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
    try:
        # Montage du système de stockage
        dbutils.fs.mount(
            source="wasbs://{}@{}.blob.core.windows.net".format(blobContainerName, storageAccountName),
            mount_point=mountPoint,
            extra_configs={'fs.azure.sas.' + blobContainerName + '.' + storageAccountName + '.blob.core.windows.net': sasToken}
        )
        print("Montage réussi !")
    except Exception as e:
        print("Exception lors du montage :", e)
else:
    print("Le point de montage existe déjà.")

# Génération de données pour chaque mois
for month_name, num_days in months:
    # Génération de données pour le mois spécifique
    start_date = datetime(2023, months.index((month_name, num_days)) + 1, 1)
    end_date = start_date + timedelta(days=num_days - 1)
    date_generated = [start_date + timedelta(days=x) for x in range(0, num_days)]

    transport_types = ["Bus", "Train", "Tram", "Metro"]
    routes = ["Route_" + str(i) for i in range(1, 11)]
    stations = ["Station_" + str(i) for i in range(1, 21)]

    # Sélection aléatoire de 5 jours comme jours de conditions météorologiques extrêmes
    extreme_weather_days = random.sample(date_generated, 5)

    data = []

    for date in date_generated:
        for _ in range(32): # 32 enregistrements par jour pour obtenir un total de 992 enregistrements pour chaque mois
            transport = random.choice(transport_types)
            route = random.choice(routes)

            # Heures de fonctionnement normales
            departure_hour = random.randint(5, 22)
            departure_minute = random.randint(0, 59)

            # Introduction d'heures de fonctionnement inhabituelles pour les bus
            if transport == "Bus" and random.random() < 0.05:  # 5% chance
                departure_hour = 3

            departure_time = f"{departure_hour:02}:{departure_minute:02}"

            # Durée normale
            duration = random.randint(10, 120)

            # Introduction de retournements courts
            if random.random() < 0.05:  # 5% chance
                duration = random.randint(1, 5)

            # Retard général
            delay = random.randint(0, 15)

            # Impact météorologique
            if date in extreme_weather_days:
                # Augmentation du retard de 10 à 60 minutes
                delay += random.randint(10, 60)

                # 10% de chance de changer l'itinéraire
                if random.random() < 0.10:
                    route = random.choice(routes)

            total_minutes = departure_minute + duration + delay
            arrival_hour = departure_hour + total_minutes // 60
            arrival_minute = total_minutes % 60

            # Assurer que les heures d'arrivée ne dépassent pas 24:59
            if arrival_hour >= 24:
                arrival_hour %= 24

            arrival_time = f"{arrival_hour:02}:{arrival_minute:02}"

            passengers = random.randint(1, 100)
            departure_station = random.choice(stations)
            arrival_station = random.choice(stations)

            data.append([date, transport, route, departure_time, arrival_time, passengers, departure_station, arrival_station, delay])

    # Création d'un DataFrame Spark à partir des données
    df = spark.createDataFrame(data, schema=["Date", "TransportType", "Route", "DepartureTime", "ArrivalTime", "Passengers", "DepartureStation", "ArrivalStation", "Delay"])
    
    # Enregistrement du DataFrame au format CSV en utilisant le point de montage
    file_path = f"/dbfs/mnt/public-transport-data/raw/public_transport_data_{month_name}.csv"
    data = df.toPandas()
    data.to_csv(file_path, index=False)
    print("Terminé")

# Démontage du lac de données
dbutils.fs.unmount(mountPoint)
